<a href="https://colab.research.google.com/github/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm/blob/oscar1/notebooks/how_to_use_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wandb Start To Track This Run

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 166 kB 37.1 MB/s 
     |████████████████████████████████| 182 kB 42.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 166 kB 50.7 MB/s 
     |████████████████████████████████| 162 kB 48.7 MB/s 
     |████████████████████████████████| 162 kB 69.9 MB/s 
     |████████████████████████████████| 158 kB 25.1 MB/s 
     |████████████████████████████████| 157 kB 75.0 MB/s 
     |████████████████████████████████| 157 kB 45.4 MB/s 
     |████████████████████████████████| 157 kB 46.7 MB/s 
     |████████████████████████████████| 157 kB 68.4 MB/s 
     |████████████████████████████████| 157 kB 58.0 MB/s 
     |████████████████████████████████| 157 kB 48.2 MB/s 
     |████████████████████████████████| 157 kB 54.2 MB/s 
     |████████████████████████████

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: os2000os (dlf22_mini_project). Use `wandb login --relogin` to force relogin


# Run Model

In [ ]:
!git clone -b main "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 289 (delta 99), reused 108 (delta 69), pack-reused 134
Receiving objects: 100% (289/289), 85.61 KiB | 5.71 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [ ]:
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

In [ ]:
import torch
import torch.nn as nn

import time
import random

In [ ]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_transforms

BATCH_SIZE = 512
VALID_RATIO = 0.1

train_data, valid_data, test_data = \
get_transformed_data(make_transforms=make_transforms, valid_ratio=VALID_RATIO)

train_iter, valid_iter, test_iter = \
make_data_loaders(train_data, valid_data, test_data, BATCH_SIZE)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

architecture = [
    (1, 64, 0.5),
    (1, 128, 0.5),
    (1, 256, 0.5),
    (1, 512, 0.5),
]

config = StemConfig(num_channels=64, kernel_size=3, stride=1, padding=1)
model  = ResNet(architecture, stem_config=config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
# intialize a new model

inputs = torch.empty((BATCH_SIZE, 3, 32, 32))
inputs.normal_()

model = model.to(device)

outputs = model(inputs.to(device))
print(outputs.size())

torch.Size([512, 10])


In [ ]:
model.apply(initialize_parameters)

ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
  (body): Sequential(
    (block_2): Sequential(
      (0): ResidualBlock(
        (dropout): Dropout(p=0.5, inplace=False)
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (out): ReLU(inplace=True)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (block_3): Sequential(
      (0

In [ ]:
print(f"num params: {sum([p.numel() for p in model.parameters() if p.requires_grad]):,}")

num params: 4,904,330


In [ ]:
from src.engine import train_one_epoch, evaluate

best_loss = float('inf')
EPOCHS  = 3
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# setup wandb logging
import wandb


ENTITY_NAME  = 'dlf22_mini_project' # this is our team name
PROJECT_NAME = 'Junk_Test_Project'
RUN_NAME     = 'resnet_oscar_test_run'

WANDB_CONFIG = \
{"learning_rate":learning_rate,
 "epochs": EPOCHS,
 "batch_size": BATCH_SIZE,
 "architecture": architecture}


wandb\
.init(name=RUN_NAME, project=PROJECT_NAME, entity=ENTITY_NAME,config=WANDB_CONFIG)